In [1]:
import glob
import os
import pandas as pd
import xarray as xr
import numpy as np
import rioxarray as rxr
from matplotlib import pyplot as plt
import geopandas as gpd

In [2]:
gdf = gpd.read_file("sf_socal-baja/")
gdf = gdf.to_crs(crs = "EPSG:4326")
gdf = gdf[gdf.US_L3CODE == '85']
gdf = gdf[gdf.Shape_Area == gdf.Shape_Area.max()]

bbox = gdf.total_bounds
latmin = bbox[1] - 1
latmax = bbox[3] + 1
lonmin = bbox[0] - 1
lonmax = bbox[2] + 1

print(latmin, latmax)
print(lonmin, lonmax)

31.534155716301562 35.51573667175686
-121.45457486710173 -115.19969953245605


----

#### DSR timeseries:

In [3]:
from shapely.geometry import Polygon

In [4]:
paths = glob.glob('/rds/general/user/cb2714/projects/wwa/ephemeral/'+
                  'cali-wildfires/cordex-fwi/DSR*.nc')

In [7]:
for path in paths:
    dsr = xr.open_dataset(path)['dsr']
    dsr = dsr.rio.write_crs("EPSG:4326")
    dsr = dsr.rio.clip(gdf.geometry.values, gdf.crs, all_touched = True)

    gdf = gpd.read_file("sf_socal-baja/")
    gdf = gdf.to_crs(crs = "EPSG:4326")
    gdf = gdf[gdf.US_L3CODE == '85']
    gdf = gdf[gdf.Shape_Area == gdf.Shape_Area.max()]

    lats = dsr.latitude.data
    lons = dsr.longitude.data

    # Testing all widths are the same:
    assert len(set(lats[1:] - lats[:-1])) == 1
    assert len(set(lons[1:] - lons[:-1])) == 1

    lat_width = np.abs(lats[1] - lats[0]) / 2
    lon_width = np.abs(lons[1] - lons[0]) / 2

    area_weights = np.zeros_like(dsr[0,:,:])
    for i,lat in enumerate(lats):
        for j,lon in enumerate(lons):
            cell = Polygon([(lon + lon_width, lat + lat_width),
                            (lon - lon_width, lat + lat_width),
                            (lon - lon_width, lat - lat_width),
                            (lon + lon_width, lat - lat_width)])
            area_weights[i,j] = cell.intersection(gdf.geometry).area / cell.area

    weighted_dsr = dsr * area_weights

    data = weighted_dsr.rolling(time = 1).mean()
    data = data.mean(dim = ['latitude','longitude'])
    data = data.resample(time = 'M').max()
    d1 = data[11::12]
    d2 = data[12::12]
    d2 = d2.sel(time = d2.time.dt.year >= d1.time.dt.year.min()+1)
    d1 = d1.sel(time = d1.time.dt.year <= d2.time.dt.year.max()-1)
    d1.data = np.max(np.array([d1.data,d2.data]), axis = 0)
    outpath = 'cordex/DSR_1day_DJ_' + path.split('/')[-1][4:-3] + '.csv'
    pd.DataFrame(d1.to_pandas(), columns = ['DJ_DSR_1day']).to_csv(outpath)

    data = weighted_dsr.rolling(time = 2).mean()
    data = data.mean(dim = ['latitude','longitude'])
    data = data.resample(time = 'M').max()
    d1 = data[11::12]
    d2 = data[12::12]
    d2 = d2.sel(time = d2.time.dt.year >= d1.time.dt.year.min()+1)
    d1 = d1.sel(time = d1.time.dt.year <= d2.time.dt.year.max()-1)
    d1.data = np.max(np.array([d1.data,d2.data]), axis = 0)
    outpath = 'cordex/DSR_2day_DJ_' + path.split('/')[-1][4:-3] + '.csv'
    pd.DataFrame(d1.to_pandas(), columns = ['DJ_DSR_2day']).to_csv(outpath)

    data = weighted_dsr.rolling(time = 3).mean()
    data = data.mean(dim = ['latitude','longitude'])
    data = data.resample(time = 'M').max()
    d1 = data[11::12]
    d2 = data[12::12]
    d2 = d2.sel(time = d2.time.dt.year >= d1.time.dt.year.min()+1)
    d1 = d1.sel(time = d1.time.dt.year <= d2.time.dt.year.max()-1)
    d1.data = np.max(np.array([d1.data,d2.data]), axis = 0)
    outpath = 'cordex/DSR_3day_DJ_' + path.split('/')[-1][4:-3] + '.csv'
    pd.DataFrame(d1.to_pandas(), columns = ['DJ_DSR_3day']).to_csv(outpath)

In [9]:
paths = glob.glob('/rds/general/user/cb2714/projects/wwa/ephemeral/'+
                  'cali-wildfires/cordex-fwi/FWI*.nc')

In [10]:
for k,path in enumerate(paths):
    print(f'{k+1} of {len(paths)}')
    fwi = xr.open_dataset(path)['fwi']
    fwi = fwi.rio.write_crs("EPSG:4326")
    fwi = fwi.rio.clip(gdf.geometry.values, gdf.crs, all_touched = True)

    gdf = gpd.read_file('/rds/general/user/cb2714/home/wwa_project/la/shapefiles/'+
                        'us_eco_l3.shp')
    gdf = gdf.to_crs(crs = "EPSG:4326")
    gdf = gdf[gdf.US_L3CODE == '85']
    gdf = gdf[gdf.Shape_Area == gdf.Shape_Area.max()]

    lats = fwi.latitude.data
    lons = fwi.longitude.data

    # Testing all widths are the same:
    assert len(set(lats[1:] - lats[:-1])) == 1
    assert len(set(lons[1:] - lons[:-1])) == 1

    lat_width = np.abs(lats[1] - lats[0]) / 2
    lon_width = np.abs(lons[1] - lons[0]) / 2

    area_weights = np.zeros_like(fwi[0,:,:])
    for i,lat in enumerate(lats):
        for j,lon in enumerate(lons):
            cell = Polygon([(lon + lon_width, lat + lat_width),
                            (lon - lon_width, lat + lat_width),
                            (lon - lon_width, lat - lat_width),
                            (lon + lon_width, lat - lat_width)])
            area_weights[i,j] = cell.intersection(gdf.geometry).area / cell.area

    weighted_fwi = fwi * area_weights

    data = weighted_fwi.rolling(time = 1).mean()
    data = data.mean(dim = ['latitude','longitude'])
    data = data.resample(time = 'M').max()
    data = data.sel(time = data.time.dt.month == 1)

    outpath = 'cordex/FWI_1day_J_' + path.split('/')[-1][4:-3] + '.csv'
    pd.DataFrame(data.to_pandas(), columns = ['J_FWI_1day']).to_csv(outpath)
    
    pd.DataFrame(weighted_fwi.mean(dim = ['latitude','longitude']).to_pandas(),
                 columns = ['fwi']).to_csv(
        (f'/rds/general/user/cb2714/projects/wwa/ephemeral/'+
         f'cali-wildfires/timeseries/daily_fwi_{path.split("/")[-1][4:-3]}.csv'))

1 of 26
2 of 26
3 of 26
4 of 26
5 of 26
6 of 26
7 of 26
8 of 26
9 of 26
10 of 26
11 of 26
12 of 26
13 of 26
14 of 26
15 of 26
16 of 26
17 of 26
18 of 26
19 of 26
20 of 26
21 of 26
22 of 26
23 of 26
24 of 26
25 of 26
26 of 26
